# Initialization

In [1]:
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics, GLPK
baltic_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_Baltic.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
baltic_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_Baltic.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))

# commodities from source to terminal node
commodities = []
for i in 1:size(baltic_demand, 1)
    push!(commodities, string(baltic_demand.Origin[i], "s", "|", baltic_demand.Destination[i], "t"))
end
;

# Services

In [3]:
# services
services = [
    [200000 1 "Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
    [130000 1 "Feeder_450" "DEBRV" "RUKGD" "FIKTK" "FIRAU" "PLGDY" "DEBRV"],
    [97000 1 "Feeder_450" "DEBRV" "DKAAR" "SEGOT" "NOKRS" "DEBRV"],
    [100000 1 "Feeder_800" "DEBRV" "SEGOT" "RULED" "DEBRV"]
]

# unique ports
service_ports = []
for service in services
    for port in service[4:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports)
;

# Port modifications

In [4]:
# List of ports that have intersecting service
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[4:end]...] for row in services][i], [[row[4:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j)); # Collecting unique transshipment nodes

# Adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i)) # vi behøves i realiteten ikke at update vores serivces
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes)

# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1]) + ports[ports.UNLocode .== port, 12][1]])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# Transshipment nodes adjacency list
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])]) # Skal vi også have move omkostninger med, da vi skal load og disch?
    end
end

# Service ports adjacency list
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 5:length(services_w_transshipment[service]) # Hvad gør vi med port call omkostninger?
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], 0.0]) # ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 12][1]]
    end
end

# Penalty edges adjacency list
penalty_adj = []
for k in 1:size(baltic_demand,1)
    push!(penalty_adj, [string(baltic_demand.Origin[k], "s"), string(baltic_demand.Destination[k], "t"), 1000])
end

dummy_adj = []
for port in 1:length(service_ports)
    push!(dummy_adj, [string(service_ports[port], "t"), nothing, 0.0])
end

edges = vcat(ports_adj, s_t_adj, transshipment_adj, penalty_adj, dummy_adj)
;

### Dict of adjacency list

In [5]:
function get_G(edges)
    # Create an empty adjacency list
    graph = Dict{String, Vector{Tuple{Any, Float64}}}()  # Use Tuple to store (target, VC)
    #graph = Dict{String, Vector{Tuple{String, Float64}}}()  # Use Tuple to store (target, VC)

    # Populate the adjacency list
    for edge in edges
        source, target, weight = edge
        if haskey(graph, source)
            push!(graph[source], (target, weight))
        else
            graph[source] = [(target, weight)]
        end
    end

    return graph
end

get_G (generic function with 1 method)

### Dijkstra's shortest path algorithm

In [6]:
function dijkstra(G, source, destination)
    dist = Dict{Any, Float64}()
    prev = Dict{Any, Any}()
    Q = Set(keys(G))

    for v in Q
        dist[v] = Inf
        prev[v] = nothing
    end

    dist[source] = 0.0

    while !isempty(Q)
        min_value = Inf
        min_key = string()
        for i in Q
            if dist[i] <= min_value
                min_value = dist[i]
                min_key = i
            end
        end
        u = min_key

        if dist[u] == Inf
            break
        end
        if u == destination
            break
        end
        
        delete!(Q, u)

        for v in G[u]
            if haskey(G, v[1])
                alt = dist[u] + v[2]
                if alt < dist[v[1]]
                    dist[v[1]] = alt
                    prev[v[1]] = u
                end
            else
                break
            end
        end
    end

    S = []
    u = destination
    push!(S, u)

    while prev[u] != nothing
        push!(S, u)
        u = prev[u]
    end
    push!(S, source)
    
    return reverse(S[2:end]), dist[destination]
end


dijkstra (generic function with 1 method)

# Flowproblem

In [7]:
current_paths = []
current_costs = []
current_commodities = []
;

### Initializing master problem

In [8]:
# Bases med penalty edges
for i in 1:length(penalty_adj)
    push!(current_paths, penalty_adj[i][1:2])
    push!(current_costs, penalty_adj[i][3])
    append!(current_commodities, i)
end


# Edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (ports_adj)
        push!(i_j, string(edge[1], "|", edge[2]))
    end
end

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_p = size(current_paths, 1) # Number of paths
P = 1:n_p

n_mv = size(baltic_fleet, 1)
v = 1:n_mv

# Initial base including penalty edges
PK = Matrix{Float64}(undef, n_k, 0) # Type of commodity for each path
mvs = Matrix{Float64}(undef, n_mv, 0) # Type of vessel on each commodity
aij = Matrix{Float64}(undef, n_ij, 0)
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0

# Constraint matrices for commodidty paths PK and edges used
for i in P
    ak = zeros(Float64, n_ij)
    pk = zeros(Float64, n_k)
    for j in 2:length(current_paths[i])
        if string(current_paths[i][j-1], "|", current_paths[i][j]) ∈ i_j
            ak[findall(x -> x == string(current_paths[i][j-1], "|", current_paths[i][j]), i_j)[1]] = 1
        end
    end
    pk[findall(x -> x == string(current_paths[i][1], "|", current_paths[i][end]), commodities)[1]] = 1
    aij = [aij ak]
    PK = [PK pk]
end

# Constraint matrix for edge capacities
for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end

# Constraint matrix for service vessels
for i in services
    mv = zeros(Float64, n_mv)
    mv[findall(x -> x == i[3], baltic_fleet."Vessel class")[1]] = 1
    mvs = [mvs mv]
end

P = 1:length(current_paths)
A = 1:size(aij, 2)
;

### Master problem

In [9]:
flowModel = Model(GLPK.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
optimize!(flowModel)
solution_summary(flowModel)

dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K] # Dual variables for commodities at iteration 1 
dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij] # Dual variables for edges at iteration 1 
reduced_cost = [-1.0] # Kickstarting while loop
;


In [10]:
while any(reduced_cost .< 0)

    append!(dual_Y_ij, zeros(Float64, length(edges) - length(ports_adj)))

    # Initializing feasible paths
    candidate_paths = []
    candidate_costs = []
    candidate_commodities = []

    # Cost for shortest path
    cij = []

    # Setting up new network with reduced cost
    edges_w_dual = deepcopy(edges)
    for i in 1:length(edges_w_dual)
        edges_w_dual[i][3] = edges_w_dual[i][3] - dual_Y_ij[i]
    end

    for i in 1:size(baltic_demand, 1)
        c = 0
        path, cost = dijkstra(get_G(edges_w_dual), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
        for j in 2:length(path)
            for edge in edges
                if edge[1] == path[j-1] && edge[2] == path[j]
                    c += edge[3]
                end
            end
        end
        push!(candidate_costs, c)
        #lav et for loop der tjekker omkostningen fra i,j i path
        push!(candidate_paths, path)
        push!(cij, cost)
        append!(candidate_commodities, i)
    end

    # bevar candidate cost da vi bruger den her, men costen i vores model/objektfunktion skal være de originale omkostninger fra 'edges'
    reduced_cost = cij - dual_Y_k - baltic_demand[:,4]


    cp_r_k = candidate_paths[reduced_cost .< 0] #candidate_paths[reduced_cost .== minimum(reduced_cost)]

    for i in 1:length(cp_r_k)
        ak = zeros(Float64, n_ij)
        pk = zeros(Float64, n_k)
        for j in 2:length(cp_r_k[i])
            if string(cp_r_k[i][j-1], "|", cp_r_k[i][j]) ∈ i_j
                ak[findall(x -> x == string(cp_r_k[i][j-1], "|", cp_r_k[i][j]), i_j)[1]] = 1
            end
        end
        pk[findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1]] = 1
        aij = [aij ak]
        PK = [PK pk]
        push!(current_paths, cp_r_k[i])
        push!(current_costs, candidate_costs[reduced_cost .< 0][i])
        append!(current_commodities, findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1])
    end

    # Cost på ij skal være fra originalt netværk (vi skal ikke have reducd cost med)

    P = 1:length(current_paths)
    n_p = length(current_paths)
    A = 1:size(aij, 2)

    revenue = zeros(Int64, n_k)
    append!(revenue, [baltic_demand[current_commodities[i],4][1] for i in n_k+1:n_p])

    flowModel = Model(GLPK.Optimizer)
    @variable(flowModel, x[P] >= 0)
    @variable(flowModel, 0 <= y[S] <= 1)
    @objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum((current_costs[j]-revenue[j])*x[j] for j in P))
    @constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
    @constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
    @constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
    optimize!(flowModel)
    # print(flowModel)

    dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K]
    dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij]

end
    # primal_x = value.(x[P])
    # primal_y = value.(y[S])


In [11]:
objective_value(flowModel)

-649559.5555555555